In [1]:
import random
from itertools import permutations
from pulp import LpProblem, LpVariable, LpStatus, LpMinimize, lpSum

def generate_dummy_data(n_pois: int, n_days: int) -> tuple:
    """
    Generate dummy data for the itinerary optimization problem.

    Parameters:
    - n_pois: The number of points of interest (POIs).
    - n_days: The number of days.

    Returns:
    - pois: A list of POI names.
    - days: A list of day identifiers.
    - travel_time: A dictionary with travel times between each pair of POIs.
    """
    pois = [f'poi_{i}' for i in range(n_pois)]
    days = [f'D{day}' for day in range(1, n_days + 1)]
    travel_time = {(poi1, poi2): random.randint(10, 60) for poi1, poi2 in permutations(pois, 2)}
    return pois, days, travel_time




In [12]:
from typing import Dict, List, Tuple
\
def extract_itinerary(nodes):
    # Step 1: Group nodes by day
    day_edges = {}
    for node in nodes:
        poi1, poi2, day = node
        if day not in day_edges:
            day_edges[day] = []
        day_edges[day].append((poi1, poi2))
    
    itineraries = {}
    # Step 2: Construct the graph for each day and find the path
    for day, edges in day_edges.items():
        graph = {}
        in_degree = {}  # Track incoming edges
        for poi1, poi2 in edges:
            graph[poi1] = poi2
            # Update in_degree counts
            in_degree[poi2] = in_degree.get(poi2, 0) + 1
            if poi1 not in in_degree:
                in_degree[poi1] = 0
        
        # Step 3: Find the starting POI (no incoming edges) and construct the itinerary
        start_poi = None
        for poi, in_deg in in_degree.items():
            if in_deg == 0:
                start_poi = poi
                break
        
        itinerary = [start_poi]
        while start_poi in graph:
            next_poi = graph[start_poi]
            itinerary.append(next_poi)
            start_poi = next_poi
        
        itineraries[day] = itinerary
    
    return itineraries

In [8]:
# Initialize data
n_pois = 15  # Number of POIs
n_days = 4  # Number of days
pois, days, travel_time = generate_dummy_data(n_pois, n_days)

# Initialize the problem
problem = LpProblem("Itinerary_Optimization", LpMinimize)

# Variables
order_vars = {(poi1, poi2, day): LpVariable(f"order_{poi1}_{poi2}_{day}", cat="Binary")
              for poi1 in pois for poi2 in pois for day in days if poi1 != poi2}

# Objective function: Minimize the total travel time
problem += lpSum(travel_time[(poi_from, poi_to)] * order_vars[(poi_from, poi_to, day)]
                 for poi_from, poi_to in permutations(pois, 2) for day in days)

# Constraints

# Each POI visited exactly once across all days
for poi in pois:
    problem += lpSum(order_vars[(poi, other_poi, day)] for day in days for other_poi in pois if poi != other_poi) == 1, f"enter_{poi}"
    problem += lpSum(order_vars[(other_poi, poi, day)] for day in days for other_poi in pois if poi != other_poi) == 1, f"exit_{poi}"

# Ensure entering and exiting each POI occurs within the same day
for day in days:
    for poi in pois:
        problem += lpSum(order_vars[(poi, other_poi, day)] for other_poi in pois if poi != other_poi) == \
                   lpSum(order_vars[(other_poi, poi, day)] for other_poi in pois if poi != other_poi), f"same_day_visit_{poi}_{day}"

# New Constraint: Ensure each day has at least one POI visited
for day in days:
    problem += lpSum(order_vars[(poi1, poi2, day)] for poi1 in pois for poi2 in pois if poi1 != poi2) >= 2, f"at_least_one_poi_{day}"
from pulp import GLPK_CMD

# Solve the problem
status = problem.solve(GLPK_CMD())

# Check the status and print the results
if LpStatus[status] == 'Optimal':
    print("Solution Found:\n")
    results = []
    for var in order_vars:
        if order_vars[var].varValue == 1:
            results.append(var)
            print(f"{var} = {order_vars[var].varValue}")
else:
    print("No optimal solution found.")


GLPSOL--GLPK LP/MIP Solver 5.0
Parameter(s) specified in the command line:
 --cpxlp /tmp/be1189ebd02a4471bddd05bd14413527-pulp.lp -o /tmp/be1189ebd02a4471bddd05bd14413527-pulp.sol
Reading problem data from '/tmp/be1189ebd02a4471bddd05bd14413527-pulp.lp'...
94 rows, 840 columns, 4200 non-zeros
840 integer variables, all of which are binary
2660 lines were read
GLPK Integer Optimizer 5.0
94 rows, 840 columns, 4200 non-zeros
840 integer variables, all of which are binary
Preprocessing...
94 rows, 840 columns, 4200 non-zeros
840 integer variables, all of which are binary
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled
Constructing initial basis...
Size of triangular part is 74
Solving LP relaxation...
GLPK Simplex Optimizer 5.0
94 rows, 840 columns, 4200 non-zeros
      0: obj =   4.970000000e+02 inf =   7.000e+01 (8)
     70: obj =   5.075000000e+02 inf =   0.000e+00 (0)
*   210: obj =   2.380000000e+02 inf =   1.703e-14 

In [33]:
import pulp

# List available solvers
for solver in pulp.listSolvers(onlyAvailable=True):
    print(solver)


GLPK_CMD
PULP_CBC_CMD


In [24]:
results

[('poi_0', 'poi_5', 'D2'),
 ('poi_1', 'poi_4', 'D1'),
 ('poi_2', 'poi_3', 'D1'),
 ('poi_3', 'poi_1', 'D1'),
 ('poi_4', 'poi_2', 'D1'),
 ('poi_5', 'poi_6', 'D2'),
 ('poi_6', 'poi_0', 'D2')]